# Availability Management
Format de fichier : Who, Date, Available

## Internals

In [ ]:
_path=''

In [16]:
import pandas as pd
import datetime as dt
import numpy as np

In [23]:
def InitAvailability(start,end, who):
    avail = np.zeros(shape=((end-start).days,len(who)))    
    dateRange = np.arange(start,end, dtype='datetime64[D]')
    return pd.DataFrame(data=avail, columns=who,index=dateRange)    

In [232]:
def SetAvailable(avail, weekdays, who, amount):
    days = pd.DataFrame(avail.index.get_values(),columns=['date'])
    days['selected']=days.applymap(lambda x:x.isoweekday() in weekdays)
    #print days[days.selected].date
    avail.loc[days[days.selected].date,who] =amount
def SetAvailableByDate(avail,weekdays, dateFrom,dateTo, who, amount):
    days = pd.DataFrame(avail.index.get_values(),columns=['date'])
    days['selected']=days.applymap(lambda x:(x.isoweekday() in weekdays) & (x >= dateFrom) & (x<=dateTo))
    #print days[days.selected].date
    avail.loc[days[days.selected].date,who] =amount
def SetNonWorkingDay(avail,weekdays,who):
    days = pd.DataFrame(avail.index.get_values(),columns=['date'])
    days['selected']=days.applymap(lambda x:x.isoweekday() in weekdays)
    #print days[days.selected].date
    avail.loc[days[days.selected].date,who] =0
def SetOnHoliday(avail, dateFrom,dateTo,who):
    avail.loc[dateFrom:dateTo,who]=0

In [145]:
def SaveToCsv(path, avail):
    _temp=avail.copy()
    _temp['Date']=_temp.index.get_values()
    #print _temp
    _result = pd.melt(_temp,id_vars=['Date'], var_name='Who',value_name='Available')[['Who','Date','Available']]
    _result.to_csv(path_or_buf=path,index=False)

## Actuals

In [211]:
WhoList=['ACH','CGA','PTI','QPI','MWA','SLA','OGO','AHI','LZI','PM','BAU','DEL','EGA','OVA']
Actuariat = ['SLA','OGO','AHI']

In [233]:
_avail = InitAvailability(dt.datetime(2016,11,21), dt.datetime(2018,12,30),\
                          WhoList)
SetAvailableByDate(_avail,[1,2,4,5],dt.datetime(2016,11,21),dt.datetime(2016,12,30),'ACH',1)
SetAvailableByDate(_avail,[1,2,4,5],dt.datetime(2017,01,01),dt.datetime(2018,12,30),'ACH',0.4)
SetAvailableByDate(_avail,[1,2,3,4,5],dt.datetime(2016,11,21),dt.datetime(2016,12,30),'PTI',1)
SetAvailableByDate(_avail,[1,2,3,4,5],dt.datetime(2017,01,01),dt.datetime(2018,12,30),'PTI',0.4)
SetAvailable(_avail,[1,2,3,4],'BAU',1)
SetAvailable(_avail,[1,5],'QPI',1)
SetAvailable(_avail,[1,2,4,5],'MWA',1)
SetAvailable(_avail,[1,2,4,5],'SLA',1)
SetAvailable(_avail,[1,2,4,5],'OGO',1)
SetAvailable(_avail,[1,2,4,5],'AHI',1)
SetAvailable(_avail,[1,2,5],'LZI',1)
SetAvailable(_avail,[1,2,3,4,5],'DEL',1)
SetAvailable(_avail,[1,2,3,4,5],'EGA',1)
SetAvailable(_avail,[1,2,3,4,5],'OVA',1)
SetAvailableByDate(_avail,[1,2,3,4,5],dt.datetime(2016,11,21),dt.datetime(2016,12,30),'CGA',0.75)
SetAvailableByDate(_avail,[1,2,3,4,5],dt.datetime(2017,01,01),dt.datetime(2018,12,30),'CGA',0.15)
SetAvailableByDate(_avail,[1,2,3,4,5],dt.datetime(2016,11,21),dt.datetime(2016,12,30),'PM',0.25)
SetAvailableByDate(_avail,[1,2,3,4,5],dt.datetime(2017,01,01),dt.datetime(2018,12,30),'PM',0.35)

#Always set the holidays last!
SetOnHoliday(_avail,dt.datetime(2016,12,7),dt.datetime(2016,12,12),'CGA')
SetOnHoliday(_avail,dt.datetime(2016,12,7),dt.datetime(2016,12,12),'PM')
SetOnHoliday(_avail,dt.datetime(2016,12,21),dt.datetime(2017,01,2),'BAU')
SetOnHoliday(_avail,dt.datetime(2016,12,21),dt.datetime(2017,01,15),'ACH')
SetOnHoliday(_avail,dt.datetime(2016,12,21),dt.datetime(2017,01,2),'PTI')
SetOnHoliday(_avail,dt.datetime(2016,12,21),dt.datetime(2018,12,31),'QPI')
SetOnHoliday(_avail,dt.datetime(2016,11,20),dt.datetime(2016,12,20),'BAU')
for who in WhoList:
    SetOnHoliday(_avail,dt.datetime(2016,12,21),dt.datetime(2016,12,31),who)
for who in Actuariat:
    SetOnHoliday(_avail,dt.datetime(2017,01,01),dt.datetime(2017,01,31),who)

In [234]:
_avail[dt.datetime(2016,11,21):dt.datetime(2016,12,30)].sum()

ACH    18.0
CGA    13.5
PTI    22.0
QPI     9.0
MWA    18.0
SLA    18.0
OGO    18.0
AHI    18.0
LZI    14.0
PM      4.5
BAU     0.0
DEL    22.0
EGA    22.0
OVA    22.0
dtype: float64

In [237]:
_avail['ACH'].head(40)

2016-11-21    1.0
2016-11-22    1.0
2016-11-23    0.0
2016-11-24    1.0
2016-11-25    1.0
2016-11-26    0.0
2016-11-27    0.0
2016-11-28    1.0
2016-11-29    1.0
2016-11-30    0.0
2016-12-01    1.0
2016-12-02    1.0
2016-12-03    0.0
2016-12-04    0.0
2016-12-05    1.0
2016-12-06    1.0
2016-12-07    0.0
2016-12-08    1.0
2016-12-09    1.0
2016-12-10    0.0
2016-12-11    0.0
2016-12-12    1.0
2016-12-13    1.0
2016-12-14    0.0
2016-12-15    1.0
2016-12-16    1.0
2016-12-17    0.0
2016-12-18    0.0
2016-12-19    1.0
2016-12-20    1.0
2016-12-21    0.0
2016-12-22    0.0
2016-12-23    0.0
2016-12-24    0.0
2016-12-25    0.0
2016-12-26    0.0
2016-12-27    0.0
2016-12-28    0.0
2016-12-29    0.0
2016-12-30    0.0
Name: ACH, dtype: float64

In [236]:
SaveToCsv('availReel.csv',_avail)